In [ ]:
import random
import math
import numpy as np
import csv

class Partition:
    def __init__(self, num_nodes, num_partitions):
        # Ensure every partition has at least one node
        base_assignment = [i % num_partitions for i in range(num_nodes)]
        random.shuffle(base_assignment)
        self.assignment = base_assignment
        self.fitness = 0

    def calculate_fitness(graph, partition, num_partitions, c_weight, b_weight):
        cut_edges = 0
        partition_sizes = [0] * num_partitions

        for u in range(len(graph)):
            for v in graph[u]:
                if u < v and partition.assignment[u] != partition.assignment[v]:
                    cut_edges += 1
            partition_sizes[partition.assignment[u]] += 1

        avg_size = len(graph) / num_partitions
        balance_penalty = sum((size - avg_size) ** 2 for size in partition_sizes)
        
        
        fitness = c_weight * cut_edges + b_weight * balance_penalty
        return fitness, cut_edges, partition_sizes

    def random_partition(num_nodes, num_partitions):
        partition = Partition(num_nodes, num_partitions)
        return partition

    def crossover(parent1, parent2):
        num_nodes = len(parent1.assignment)
        child = Partition(num_nodes, len(set(parent1.assignment)))
        
        crossover_point = random.randint(0, num_nodes - 1)
        
        child.assignment[:crossover_point] = parent1.assignment[:crossover_point]
        child.assignment[crossover_point:] = parent2.assignment[crossover_point:]
        
        return child

    def mutate(partition, num_partitions):
        node = random.randint(0, len(partition.assignment) - 1)
        old_partition = partition.assignment[node]
        new_partition = random.randint(0, num_partitions - 1)
        partition.assignment[node] = new_partition
        print(f"Mutated node {node} from partition {old_partition} to partition {new_partition}")

    def selection(population):
        idx1, idx2 = random.sample(range(len(population)), 2)
        return population[idx1] if population[idx1].fitness < population[idx2].fitness else population[idx2]

    def load_graph(filename):
        with open(filename, 'r') as f:
            num_nodes, num_edges = map(int, f.readline().split())
            graph = [[] for _ in range(num_nodes)]
            for _ in range(num_edges):
                u, v = map(int, f.readline().split())
                graph[u].append(v)
                graph[v].append(u)
        return graph

    def save_to_csv(data, filename, include_header=True):
        header = ['Generation', 'Num Partitions', 'Best Fitness', 'Cut Edges', 'Max-Min Partition Size Difference','c_weight','b_weight']
        mode = 'a' if not include_header else 'w'
        
        with open(filename, mode, newline='') as f:
            writer = csv.writer(f)

            if include_header:
                writer.writerow(header)

            writer.writerows(data)


    def genetic_algorithm(graph, num_partitions, population_size, generations,c_weight, b_weight):
        population = []

        csv_data = []

        for _ in range(population_size):
            p = random_partition(len(graph), num_partitions)
            p.fitness, _, _ = calculate_fitness(graph, p, num_partitions,c_weight,b_weight)
            population.append(p)

        for gen in range(generations):
            new_population = []
            
            for _ in range(population_size):
                parent1 = selection(population)
                parent2 = selection(population)

                child = crossover(parent1, parent2)
                if random.random() < 0.2:
                    mutate(child, num_partitions)

                child.fitness, cut_edges, partition_sizes = calculate_fitness(graph, child, num_partitions,c_weight, b_weight)
                new_population.append(child)

            population = new_population

            # Get the best partition and record statistics
            best_partition = min(population, key=lambda p: p.fitness)
            best_fitness, best_cut_edges, partition_sizes = calculate_fitness(graph, best_partition, num_partitions,c_weight, b_weight)
            max_min_diff = max(partition_sizes) - min(partition_sizes)

            print(f"Generation {gen + 1}: Best Fitness = {best_fitness}, Cut Edges = {best_cut_edges}, Partition Size Difference = {max_min_diff}")
            print(f"Best partition assignments: {best_partition.assignment}")

            # Save data for CSV
            csv_data.append([gen + 1, num_partitions, best_fitness, best_cut_edges, max_min_diff,c_weight,b_weight])

        return min(population, key=lambda p: p.fitness), csv_data



def main():
    filename = "graph.txt"  
    partition_sizes = [10,30,50,80,100]  
    population_size = 6  
    generations = 5000

    graph = load_graph(filename)
    output_csv = "dense_graph_data_1_ga.csv"

    all_csv_data = [] 

    
    weight_pairs = [(4,1), (1,3), (1,1)]

    include_header = True 

    for c_weight, b_weight in weight_pairs:
        print(f"\nRunning genetic algorithm with c_weight={c_weight}, b_weight={b_weight}")

        for num_partitions in partition_sizes:
            print(f"\nRunning genetic algorithm with {num_partitions} partitions...")     
            best_partition, csv_data = genetic_algorithm(graph, num_partitions, population_size, generations, c_weight, b_weight)
            all_csv_data.extend(csv_data)

            print(f"\nBest partition fitness for {num_partitions} partitions: {best_partition.fitness}")
            for i, partition in enumerate(best_partition.assignment):
                print(f"Node {i} -> Partition {partition}")


    save_to_csv(all_csv_data, output_csv, include_header=include_header)

if __name__ == "__main__":
    main()




Running genetic algorithm with c_weight=4, b_weight=1

Running genetic algorithm with 10 partitions...
Mutated node 378 from partition 0 to partition 8
Mutated node 398 from partition 3 to partition 6
Generation 1: Best Fitness = 125952.9, Cut Edges = 31411, Partition Size Difference = 18
Best partition assignments: [6, 8, 9, 6, 4, 7, 1, 9, 3, 9, 1, 6, 6, 5, 5, 2, 7, 6, 4, 8, 2, 7, 5, 9, 0, 0, 1, 2, 8, 5, 2, 6, 9, 7, 2, 8, 7, 6, 9, 3, 5, 3, 8, 4, 3, 7, 1, 4, 9, 8, 8, 2, 9, 4, 2, 8, 0, 7, 3, 8, 3, 8, 5, 0, 7, 2, 8, 1, 0, 4, 4, 3, 5, 9, 1, 2, 5, 6, 1, 1, 1, 1, 5, 7, 5, 5, 8, 8, 5, 0, 3, 9, 9, 8, 6, 8, 1, 9, 0, 7, 0, 5, 1, 4, 5, 1, 5, 5, 2, 6, 0, 2, 2, 2, 4, 5, 6, 4, 8, 4, 1, 5, 2, 8, 8, 0, 2, 4, 2, 0, 8, 3, 2, 6, 1, 6, 0, 0, 1, 9, 8, 9, 2, 7, 2, 8, 1, 0, 9, 4, 6, 5, 2, 8, 3, 3, 4, 0, 6, 7, 6, 8, 2, 0, 7, 8, 4, 6, 2, 5, 4, 5, 5, 0, 2, 8, 8, 1, 1, 4, 9, 9, 3, 8, 6, 4, 2, 3, 8, 2, 0, 2, 8, 3, 5, 6, 3, 6, 3, 1, 6, 5, 1, 9, 9, 5, 1, 2, 1, 4, 5, 7, 7, 4, 9, 6, 5, 0, 5, 2, 0, 2, 3, 4, 4, 9, 1,